## Подготовка модели прогнозирования ...

Загрузка необходимых библиотек

In [1]:
import numpy as np
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV
import pandas as pd
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
import matplotlib.pyplot as plt
%matplotlib inline
simplefilter("ignore", category=ConvergenceWarning)
pd.options.mode.chained_assignment = None

### Предварительная подготовка данных

Загрузка датасета

In [2]:
df = pd.read_excel('db.xls')

Разделим датасет на части по ...

In [3]:
df_3 = df.loc[1:30,:]
df_2 = df.loc[32:62,:]
df_1 = df.loc[64:94,:]
df_0 = df.loc[96:,:]

Введем значение целевой переменной в соответствии с описанием заказчика.

In [4]:
df_3['y'] = 1
df_2['y'] = 1
df_1['y'] = 0
df_0['y'] = 0

Соеденим обратно датафреймы с целевой переменной в общий датафрейм

In [5]:
data = pd.concat([df_3, df_2, df_1, df_0], axis=0).reset_index(drop=True)

Проверим пропуски

In [ ]:
data.isna().sum()

Обнаружены пропуски, способы их заполнения обсуждены с заказчиков и также столбец с текстом приведен к формату приемлемому для мл.

Поанализируем библиотекой ydata-profiling

In [110]:
profile = ProfileReport(data, title="Report")

In [ ]:
profile.to_file("eda_for_bd.html")

Анализ результата (записан в файл "eda_for_bd.html") показывает наличие корреляций целевой с переменной с предикторами (хоть и не со всеми).

### Построение модели прогнозирования

Разделим датасет на тренировочную и тестовую выборки с использованием стратификации по целевой переменной

In [43]:
X = data.drop(['y'], axis=1)
y = data[['y']]['y'].ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=333, stratify=y)

Применим известные модели классификации с небольшим тюнингом GridSearchCV с метрикой классификации roc_auc и нессиметричной метрикой fbeta_score в которой полнота в 2 раза важнее точности

Соберем метрику ftwo_scorer (2-я описанная выше)

In [44]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

### Логистическая регрессия

Модель по метрике roc_auc

In [45]:
params_logreg = {'solver':('liblinear', 'saga'), 'C':[0.5, 0.8, 1.0, 1.2, 1.5, 2.0]}
model_logreg = LogisticRegression(random_state=12345, class_weight='balanced', penalty='l1', max_iter=1000)

grid_logreg = GridSearchCV(model_logreg, params_logreg, cv=3, verbose=0, scoring='roc_auc')
grid_logreg.fit(X_train, y_train)
model_logreg = grid_logreg.best_estimator_
print(f'best roc_auc for train {grid_logreg.best_score_}')
print(f'roc_auc test = {roc_auc_score(y_test, model_logreg.predict(X_test))}')
print(f'вероятность правильного ответа test = {accuracy_score(y_test, model_logreg.predict(X_test))}')
print(f'точность test = {precision_score(y_test, model_logreg.predict(X_test))}')
print(f'полнота test = {recall_score(y_test, model_logreg.predict(X_test))}')

best roc_auc for train 0.9955065359477123
roc_auc test = 0.9464285714285715
вероятность правильного ответа test = 0.925
точность test = 0.8
полнота test = 1.0


Модель с нессиметричной метрикой

In [46]:
params_logreg = {'solver':('liblinear', 'saga'), 'C':[0.5, 0.8, 1.0, 1.2, 1.5, 2.0]}
model_logreg2 = LogisticRegression(random_state=12345, class_weight='balanced', penalty='l1', max_iter=1000)

grid_logreg = GridSearchCV(model_logreg2, params_logreg, cv=3, verbose=0, scoring=ftwo_scorer)
grid_logreg.fit(X_train, y_train)
model_logreg2 = grid_logreg.best_estimator_
print(f'best fb_mera for train {grid_logreg.best_score_}')
print(f'roc_auc test = {roc_auc_score(y_test, model_logreg2.predict(X_test))}')
print(f'вероятность правильного ответа test = {accuracy_score(y_test, model_logreg2.predict(X_test))}')
print(f'точность test = {precision_score(y_test, model_logreg2.predict(X_test))}')
print(f'полнота test = {recall_score(y_test, model_logreg2.predict(X_test))}')

best fb_mera for train 0.9353222996515679
roc_auc test = 0.9464285714285715
вероятность правильного ответа test = 0.925
точность test = 0.8
полнота test = 1.0


### Дерево решений

In [47]:
params_tree = {'criterion': ('gini', 'entropy', 'log_loss'), 'max_depth':range(2, 10), 
              'min_samples_leaf': range(1, 10)}
model_tree = DecisionTreeClassifier(random_state=12345)

grid_tree = GridSearchCV(model_tree, params_tree, cv=3, verbose=0, scoring='roc_auc')
grid_tree.fit(X_train, y_train)
model_tree = grid_tree.best_estimator_
print(f'best roc_auc for train {grid_tree.best_score_}')
print(f'roc_auc test = {roc_auc_score(y_test, model_tree.predict(X_test))}')
print(f'вероятность правильного ответа test = {accuracy_score(y_test, model_tree.predict(X_test))}')
print(f'точность test = {precision_score(y_test, model_tree.predict(X_test))}')
print(f'полнота test = {recall_score(y_test, model_tree.predict(X_test))}')

best roc_auc for train 0.961635348583878
roc_auc test = 0.9166666666666667
вероятность правильного ответа test = 0.95
точность test = 1.0
полнота test = 0.8333333333333334


In [48]:
params_tree = {'criterion': ('gini', 'entropy', 'log_loss'), 'max_depth':range(2, 10), 
              'min_samples_leaf': range(1, 10)}
model_tree2 = DecisionTreeClassifier(random_state=12345)

grid_tree = GridSearchCV(model_tree2, params_tree, cv=3, verbose=0, scoring=ftwo_scorer)
grid_tree.fit(X_train, y_train)
model_tree2 = grid_tree.best_estimator_
print(f'best fb_mera for train {grid_tree.best_score_}')
print(f'roc_auc = {roc_auc_score(y_test, model_tree2.predict(X_test))}')
print(f'вероятность правильного ответа = {accuracy_score(y_test, model_tree2.predict(X_test))}')
print(f'точность = {precision_score(y_test, model_tree2.predict(X_test))}')
print(f'полнота = {recall_score(y_test, model_tree2.predict(X_test))}')

best fb_mera for train 0.9265962795374559
roc_auc = 0.9821428571428572
вероятность правильного ответа = 0.975
точность = 0.9230769230769231
полнота = 1.0


В качестве лучшей модели выбрана модель дерева решений построенная на ассиметричной метрике, поскольку этой модели удалось достигнуть полноты 1.0 при более высокой точности 0.92 относительно 0.8.

Покажем значимость признаков

In [ ]:
df_feature_importances_ = pd.DataFrame([model_tree2.feature_importances_], columns=X_train.columns)

delete_columns = []
for i in df_feature_importances_.columns:
    if df_feature_importances_[i][0] == 0.0:
        delete_columns.append(i)
df_feature_importances_ = df_feature_importances_.drop(delete_columns, axis=1)
df_feature_importances_

Покажем полученное дерево принятия решения

In [ ]:
print(tree.export_text(decision_tree=model_tree2, feature_names=list(X_train.columns)))

Откорректируем порог принятия решения о принадлежности к классу 1 (болен)

In [52]:
model_tree2.predict_proba(X_test)[:, 1]

array([0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0.])

Порог откорректировать не удастся, модель уверена в своем ответе даже когда ошибается.

Проведем оценку качества полученной модели при помощи бутсрепа тестовой выборки

In [58]:
precisions = []
accuracyes = []
for _ in range(30):
    sample = np.random.choice(range(X_test.shape[0]), size=X_test.shape[0], replace=True)
    precisions.append(precision_score(y_test[sample], model_tree2.predict(X_test.iloc[sample])))
    accuracyes.append(accuracy_score(y_test[sample], model_tree2.predict(X_test.iloc[sample])))

In [59]:
print('Оценка полноты модели (доля правильных ответов модели по классу 1 среди всех ответов, где истинная метка 1)')
print(f'нижняя граница доверительного интервала {round(np.quantile(recalls, 0.025), 2)}')
print(f'среднее значение {round(np.mean(recalls), 2)}')
print(f'верхняя граница доверительного интервала {round(np.quantile(recalls, 0.975), 2)}')

Оценка полноты модели (доля правильных ответов модели по классу 1 среди всех ответов, где истинная метка 1)
нижняя граница доверительного интервала 1.0
среднее значение 1.0
верхняя граница доверительного интервала 1.0


In [60]:
print('Оценка точности модели (как много отрицательных ответов нашла модель, пока искала положительные)')
print(f'нижняя граница доверительного интервала {round(np.quantile(accuracyes, 0.025), 2)}')
print(f'среднее значение {round(np.mean(accuracyes), 2)}')
print(f'верхняя граница доверительного интервала {round(np.quantile(accuracyes, 0.975), 2)}')

Оценка точности модели (как много отрицательных ответов нашла модель, пока искала положительные)
нижняя граница доверительного интервала 0.92
среднее значение 0.98
верхняя граница доверительного интервала 1.0


### Общий вывод по точности модели:
* модель находит всех больных из тестового набора данных с 95% вероятностью
* однако, с учетом настройки на гипердиагностику имеются ложнополижтельные срабатывания (модель делает ответ 1 а на самом деле ответ 0), общая вероятность правлильного ответа составляет 0.98 (от 0.92 до 1.0)